In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
print('ready')

Init Plugin
readyInit Graph Optimizer
Init Kernel



##### This data is from Kaggle and provides information about 500 applications for various universities and what their chance of admittance is. We will build a regression model from this to predict any future admission and chance of acceptence

#### Load in dataset

In [2]:
url = 'https://raw.githubusercontent.com/vinay-jaju/EDA-and-PCA-on-Graduate-Admission-Dataset/master/input/Admission_Predict_Ver1.1.csv'
df = pd.read_csv(url)
print(df.columns)

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR ', 'CGPA', 'Research', 'Chance of Admit '],
      dtype='object')


#### Create feautures and labels

In [3]:
features = df.iloc[:,:-1]
labels = df.iloc[:,-1]
features = features.drop('Serial No.', axis=1)


In [4]:
print(features)
print(labels)

     GRE Score  TOEFL Score  University Rating  SOP  LOR   CGPA  Research
0          337          118                  4  4.5   4.5  9.65         1
1          324          107                  4  4.0   4.5  8.87         1
2          316          104                  3  3.0   3.5  8.00         1
3          322          110                  3  3.5   2.5  8.67         1
4          314          103                  2  2.0   3.0  8.21         0
..         ...          ...                ...  ...   ...   ...       ...
495        332          108                  5  4.5   4.0  9.02         1
496        337          117                  5  5.0   5.0  9.87         1
497        330          120                  5  4.5   5.0  9.56         1
498        312          103                  4  4.0   5.0  8.43         0
499        327          113                  4  4.5   4.5  9.04         0

[500 rows x 7 columns]
0      0.92
1      0.76
2      0.72
3      0.80
4      0.65
       ... 
495    0.87
496 

#### Split into training and test set

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
features_training_set, features_test_set, labels_training_set, labels_test_set = train_test_split(features, labels, 
                                                                                   test_size=0.20, random_state=42)

#### Scaling and Normalizing numerical data

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.compose import ColumnTransformer


In [8]:
scaler = ColumnTransformer([('scale', StandardScaler(), ['GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR ', 'CGPA', 'Research'])], remainder='passthrough')
#fit the scaler to the training data and test data
features_training_set_scale = scaler.fit_transform(features_training_set) 
features_test_set_scale = scaler.transform(features_test_set) 

#ColumnTransformer returns numpy arrays. Convert the features to dataframes
features_training_set_scale = pd.DataFrame(features_training_set_scale, columns = features_training_set.columns)
features_test_set_scale = pd.DataFrame(features_test_set_scale, columns = features_test_set.columns)

#### Creating a function of tensorflow model

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense

In [10]:
def regression_model(features):
    num_features = features.shape[1] 
    model = Sequential()
    input = InputLayer(input_shape = (num_features, ))
    model.add(input)
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1))
    print(model.summary())
    return model


#### Compile and fit model

In [11]:
model = regression_model(features_training_set_scale)
opt = Adam(learning_rate=0.1)

Metal device set to: Apple M1
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                512       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 577
Trainable params: 577
Non-trainable params: 0
_________________________________________________________________
None


2021-07-25 17:04:16.315540: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-07-25 17:04:16.315642: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [12]:
model.compile(loss='mse', metrics=['mae'], optimizer=opt)

In [ ]:
model.fit(features_training_set_scale, labels_training_set, epochs=40, batch_size = 1, verbose = 1)

Epoch 1/40


2021-07-25 17:04:16.420255: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-07-25 17:04:16.423120: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


#### Evaluate on testing data

In [ ]:
val_mse, val_mae = model.evaluate(features_test_set_scale, labels_test_set, verbose=0)
 
print("MAE: ", val_mae)
print("MSE: ", val_mse)

In [ ]:
from sklearn.metrics import r2_score
predicted_values = model.predict(features_test_set_scale)
r2_score(labels_test_set, predicted_values)
#R-squared score near zero showing strength of regression model